In [105]:
# Dependency, Load, Clean Data
import pandas as pd

#filepath as strings, read as DFs
sch_file = "resources\schools_complete.csv"
stu_file = "resources\students_complete.csv"
sch_df = pd.read_csv(sch_file)
stu_df = pd.read_csv(stu_file)

#clean student_name data with list, for loop, str.replace
presuffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]
for psfix in presuffixes:
    stu_df["student_name"] = stu_df["student_name"].str.replace(psfix, "")

C:\Users\nabil\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  del sys.path[0]


## Deliverable 1: Replace the reading and math scores.

### Replace the 9th grade reading and math scores at Thomas High School with NaN.

In [106]:
# Install numpy using conda install numpy or pip install numpy. 
# Step 1. Import numpy as np.
import numpy as np

In [107]:
# Step 2. use loc + logical operatiors to isolate thomas 9th graders names, change read and math grades to NaN

stu_df.loc[(stu_df["school_name"] == "Thomas High School") & (stu_df["grade"] == "9th"), "reading_score"] = np.nan


In [108]:
#  Step 3. Refactor the code in Step 2 to replace the math scores with NaN.
stu_df.loc[(stu_df["school_name"] == "Thomas High School") & (stu_df["grade"] == "9th"), "math_score"] = np.nan


In [109]:
#  Step 4. Check the student data for NaN's. 

#check Thomas Niners if their grades changed to NaN
#stu_df.loc[(stu_df["school_name"] == "Thomas High School") & (stu_df["grade"] == "9th")]

#check the entire stu_df
stu_df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66.0,79.0
1,1,Victor Smith,M,12th,Huang High School,94.0,61.0
2,2,Kevin Rodriguez,M,12th,Huang High School,90.0,60.0
3,3,Richard Scott,M,12th,Huang High School,67.0,58.0
4,4,Bonnie Ray,F,9th,Huang High School,97.0,84.0
...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99.0,90.0
39166,39166,Dawn Bell,F,10th,Thomas High School,95.0,70.0
39167,39167,Rebecca Tanner,F,9th,Thomas High School,NaN,NaN
39168,39168,Desiree Kidd,F,10th,Thomas High School,99.0,90.0


## Deliverable 2 : Repeat the school district analysis

### District Summary

In [110]:
#MERGING DATA + DISTRICT METRICS
#merge both dfs for complete data set get basic metrics
all_data = pd.merge(stu_df, sch_df, on=["school_name", "school_name"])

student_count = all_data["Student ID"].count()
school_count = len(all_data["school_name"].unique())
total_budget = sch_df["budget"].sum()
mean_read = all_data["reading_score"].mean()
mean_math = all_data["math_score"].mean()

In [124]:
# Step 1. Get the number of students that are in ninth grade at Thomas High School.
only_9_thomas_df = stu_df.loc[(stu_df["school_name"] == "Thomas High School") & (stu_df["grade"] == "9th")]
thomas_niner_count = len(only_9_thomas_df["student_name"])

#Get raw student count 
student_count = all_data["Student ID"].count()

#Get new student count, deduct thomas school ninth grades) 
student_count = student_count - thomas_niner_count

In [10]:
# Calculate Dictrict Metrics using the "clean_student_data" + new student counts.
#pass or fail into new DFs, do counts, and percentages
pass_math = all_data[all_data["math_score"] > 69]
pass_read = all_data[all_data["reading_score"] > 69]
pass_both = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]

math_pass_count = pass_math["student_name"].count()
read_pass_count = pass_read["student_name"].count()
pass_both_count = pass_both["student_name"].count()

math_perc = math_pass_count / float(student_count) * 100
read_perc = read_pass_count / float(student_count) * 100
both_perc = pass_both_count / float(student_count) * 100

In [122]:
#create district summary df
dist_sum_df = pd.DataFrame(
    [{"Total Schools" : school_count,
    "Total Students" : student_count,
    "Total Budget" : total_budget,
    "Ave Math Score" : mean_math,
    "Ave Read Score" : mean_read,
    "% Pass Math" : math_perc,
    "% Pass Read" : read_perc,
    "% Overall Pass" : both_perc}])

#format each column appropriately
dist_sum_df["Total Students"] = dist_sum_df["Total Students"].map("{:,}".format)
dist_sum_df["Ave Math Score"] = dist_sum_df["Ave Math Score"].map("{:.1f}".format)
dist_sum_df["Ave Read Score"] = dist_sum_df["Ave Read Score"].map("{:.1f}".format)
dist_sum_df["% Pass Math"] = dist_sum_df["% Pass Math"].map("{:.1f}".format)
dist_sum_df["% Pass Read"] = dist_sum_df["% Pass Read"].map("{:.1f}".format)
dist_sum_df["% Overall Pass"] = dist_sum_df["% Overall Pass"].map("{:.1f}".format)
dist_sum_df["Total Budget"] = dist_sum_df["Total Budget"].map("${:,.2f}".format)

#print output
dist_sum_df

,Total Schools,Total Students,Total Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
0,15,"38,709","$24,649,428.00",78.9,81.9,73.9,84.7,64.1


##  School Summary

In [127]:
#determine school types
per_school_types = sch_df.set_index(["school_name"])["type"]
per_school_types

#use value_counts for categorical counting
stud_per_school = stu_df["school_name"].value_counts()

#create series for basic budget table and do calculation
budget_per_school = school_types = sch_df.set_index(["school_name"])["budget"]
per_school_capita = budget_per_school/stud_per_school

#grab mean math and read per school
school_math_mean = all_data.groupby(["school_name"]).mean()["math_score"]
school_read_mean = all_data.groupby(["school_name"]).mean()["reading_score"]

#per school pass math/read create DF and count per DF 
school_pass_math = all_data[(all_data["math_score"] > 69)]
school_pass_read = all_data[(all_data["reading_score"] > 69)]
school_pass_overall = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]
school_pass_m_count = school_pass_math.groupby(["school_name"]).count()["student_name"]
school_pass_r_count = school_pass_read.groupby(["school_name"]).count()["student_name"]
school_pass_all_count = school_pass_overall.groupby(["school_name"]).count()["student_name"]

#calculate percent pass per class per school
math_pass_perc_school = school_pass_m_count / stud_per_school * 100
read_pass_perc_school = school_pass_r_count / stud_per_school * 100
overall_pass_perc_school = school_pass_all_count / stud_per_school * 100

#create  and format output DF
school_sum_df = pd.DataFrame({
    "School Type" : per_school_types,
    "Total Students" : stud_per_school,
    "Total School Budget" : budget_per_school,
    "Per Student Budget" : per_school_capita,
    "Ave Math Score" : school_math_mean,
    "Ave Read Score" : school_read_mean,
    "% Pass Math" : math_pass_perc_school,
    "% Pass Read" : read_pass_perc_school,
    "% Overall Pass" : overall_pass_perc_school})


#FORMAT LATER
#this df will be used althroughout for further wrangling
#print output


In [128]:
# Format the Total School Budget and the Per Student Budget
school_sum_df["Total School Budget"] = school_sum_df["Total School Budget"].map("${:,.2f}".format)
school_sum_df["Per Student Budget"] = school_sum_df["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
school_sum_df

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [190]:
# Step 5.  Get the number of 10th-12th graders from Thomas High School (THS).
all_thomas_df = all_data[all_data["school_name"] == "Thomas High School"]
thomas_all_count = len(all_thomas_df["student_name"])
thomas_10to12 = thomas_all_count - thomas_niner_count

In [191]:
# Step 6. Get all the students passing math from THS
ths_pass_math = all_data[(all_data["school_name"] == "Thomas High School") & (all_data["math_score"] > 69)]

In [192]:
# Step 7. Get all the students passing reading from THS
ths_pass_read = all_data[(all_data["school_name"] == "Thomas High School") & (all_data["reading_score"] > 69)]

In [193]:
# Step 8. Get all the students passing math and reading from THS
ths_pass_both = all_data[(all_data["reading_score"] > 69) & (all_data["math_score"] > 69)]
ths_pass_both = ths_pass_both[ths_pass_both["school_name"] == "Thomas High School"]

In [194]:
# Step 9. Calculate the percentage of 10th-12th grade students passing math from Thomas High School.
ths_math_perc_10to12 = len(ths_pass_math) / float(thomas_10to12) * 100
ths_math_perc_10to12

93.18568994889267

In [195]:
# Step 10. Calculate the percentage of 10th-12th grade students passing reading from Thomas High School.
ths_read_perc_10to12 = len(ths_pass_read) / float(thomas_10to12) * 100
ths_read_perc_10to12

97.01873935264055

In [196]:
# Step 11. Calculate the overall passing percentage of 10th-12th grade from Thomas High School. 
ths_both_perc_10to12 = len(ths_pass_both) / float(thomas_10to12) * 100
ths_both_perc_10to12

90.63032367972743

In [197]:
# Step 12. Replace the passing math percent for Thomas High School in the per_school_summary_df.
school_sum_df.loc[(school_sum_df["% Pass Math"] == 66.911315), "% Pass Math"] = ths_math_perc_10to12

In [198]:
# Step 13. Replace the passing reading percentage for Thomas High School in the per_school_summary_df.
school_sum_df.loc[(school_sum_df["% Pass Read"] == 69.663609), "% Pass Read"] = ths_read_perc_10to12


In [199]:
# Step 14. Replace the overall passing percentage for Thomas High School in the per_school_summary_df.
school_sum_df.loc[(school_sum_df["% Overall Pass"] == 65.076453), "% Pass Read"] = ths_read_perc_10to12

In [201]:
school_sum_df

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## High and Low Performing Schools 

In [202]:
# Sort and show top five schools.

top_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = False)

top_sch.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541
Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,93.333333,96.611111,90.333333


In [203]:
# Sort and show top five schools.

bottom_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = True)

bottom_sch.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math and Reading Scores by Grade

In [207]:
# Create a Series of scores by grade levels using conditionals.

grade9 = all_data[(all_data["grade"]== "9th")]
grade10 = all_data[(all_data["grade"] == "10th" )]
grade11 = all_data[(all_data["grade"] == "11th" )]
grade12 = all_data[(all_data["grade"] == "12th" )]

grade9_math = grade9.groupby(["school_name"]).mean()["math_score"]
grade10_math = grade10.groupby(["school_name"]).mean()["math_score"]
grade11_math = grade11.groupby(["school_name"]).mean()["math_score"]
grade12_math = grade12.groupby(["school_name"]).mean()["math_score"]

grade9_read = grade9.groupby(["school_name"]).mean()["reading_score"]
grade10_read = grade10.groupby(["school_name"]).mean()["reading_score"]
grade11_read = grade11.groupby(["school_name"]).mean()["reading_score"]
grade12_read = grade12.groupby(["school_name"]).mean()["reading_score"]

#create and format math per grade per df
math_per_grade = pd.DataFrame ({
    "9th" : grade9_math,
    "10th" : grade10_math,
    "11th" : grade11_math,
    "12th" : grade12_math})

math_per_grade["9th"] = math_per_grade["9th"].map("{:.1f}".format)
math_per_grade["10th"] = math_per_grade["10th"].map("{:.1f}".format)
math_per_grade["11th"] = math_per_grade["11th"].map("{:.1f}".format)
math_per_grade["12th"] = math_per_grade["12th"].map("{:.1f}".format)

#create and format math per grade per school df
read_per_grade = pd.DataFrame ({
    "9th" : grade9_read,
    "10th" : grade10_read,
    "11th" : grade11_read,
    "12th" : grade12_read})

read_per_grade["9th"] = read_per_grade["9th"].map("{:.1f}".format)
read_per_grade["10th"] = read_per_grade["10th"].map("{:.1f}".format)
read_per_grade["11th"] = read_per_grade["11th"].map("{:.1f}".format)
read_per_grade["12th"] = read_per_grade["12th"].map("{:.1f}".format)

# correct order, remove indices if any
math_per_grade = math_per_grade[["9th", "10th", "11th", "12th"]]
read_per_grade = read_per_grade[["9th", "10th", "11th", "12th"]]
math_per_grade.index.name = None
read_per_grade.index.name = None

#print outputs - TWO OUTPUTS
#read_per_grade

In [211]:
#Display the math ave scores per grade per school data frame
math_per_grade

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


In [ ]:
#Display the reading ave scores per grade per school data frame
read_per_grade

## Scores by School Spending

In [216]:
#School Spending as Index
#Establish the spending bins and group names.
#create bins, labels, and CUT data, add to school summary DF
spending_bins = [0,585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
school_sum_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

#school summary averages with spend grouping
spend_math_grade = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["Ave Math Score"]
spend_math_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Pass Math"]
spend_read_grade = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["Ave Read Score"]
spend_read_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Pass Read"]
spend_overall_perc = school_sum_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Pass"]

#create and format output df
spend_sum_df = pd.DataFrame({
          "Ave Math Score" : spend_math_grade,
          "Ave Read Score": spend_read_grade,
          "% Pass Math": spend_math_perc,
          "% Pass Read": spend_read_perc,
          "% Overall Pass": spend_overall_perc})

spend_sum_df["Ave Math Score"] = spend_sum_df["Ave Math Score"].map("{:.1f}".format)
spend_sum_df["Ave Read Score"] = spend_sum_df["Ave Read Score"].map("{:.1f}".format)
spend_sum_df["% Pass Math"] = spend_sum_df["% Pass Math"].map("{:.0f}".format)
spend_sum_df["% Pass Read"] = spend_sum_df["% Pass Read"].map("{:.0f}".format)
spend_sum_df["% Overall Pass"] = spend_sum_df["% Overall Pass"].map("{:.0f}".format)

In [217]:
#Output
spend_sum_df

,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93,97,90
$585-629,81.9,83.2,87,93,81
$630-644,78.5,81.6,67,77,56
$645-675,77.0,81.0,66,81,54


## Scores by School Size

In [218]:
# School Size as Index
# Establish the school size bins and labels
stud_bins = [0, 1000, 2000, 5000]
bin_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_sum_df["School Size"] = pd.cut(school_sum_df["Total Students"], stud_bins, labels=bin_labels)

#school summary averages with spend grouping
size_math_grade = school_sum_df.groupby(["School Size"]).mean()["Ave Math Score"]
size_math_perc = school_sum_df.groupby(["School Size"]).mean()["% Pass Math"]
size_read_grade = school_sum_df.groupby(["School Size"]).mean()["Ave Read Score"]
size_read_perc = school_sum_df.groupby(["School Size"]).mean()["% Pass Read"]
size_overall_perc = school_sum_df.groupby(["School Size"]).mean()["% Overall Pass"]

#create and format output df
size_sum_df = pd.DataFrame({
          "Ave Math Score" : size_math_grade,
          "Ave Read Score": size_read_grade,
          "% Pass Math": size_math_perc,
          "% Pass Read": size_read_perc,
          "% Overall Pass": size_overall_perc})

size_sum_df["Ave Math Score"] = size_sum_df["Ave Math Score"].map("{:.1f}".format)
size_sum_df["Ave Read Score"] = size_sum_df["Ave Read Score"].map("{:.1f}".format)
size_sum_df["% Pass Math"] = size_sum_df["% Pass Math"].map("{:.0f}".format)
size_sum_df["% Pass Read"] = size_sum_df["% Pass Read"].map("{:.0f}".format)
size_sum_df["% Overall Pass"] = size_sum_df["% Overall Pass"].map("{:.0f}".format)

In [219]:
#Output 
size_sum_df

,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
School Size,,,,,
Small (<1000),83.8,83.9,94,96,90
Medium (1000-2000),83.4,83.9,88,91,85
Large (2000-5000),77.7,81.3,70,83,58


## Scores by School Type

In [221]:
#School Type as Index
#No bins and labels needed, index is binary, assign value direct

type_math_grade = school_sum_df.groupby(["School Type"]).mean()["Ave Math Score"]
type_math_perc = school_sum_df.groupby(["School Type"]).mean()["% Pass Math"]
type_read_grade = school_sum_df.groupby(["School Type"]).mean()["Ave Read Score"]
type_read_perc = school_sum_df.groupby(["School Type"]).mean()["% Pass Read"]
type_overall_perc = school_sum_df.groupby(["School Type"]).mean()["% Overall Pass"]

#create and format output df
type_sum_df = pd.DataFrame({
          "Ave Math Score" : type_math_grade,
          "Ave Read Score": type_read_grade,
          "% Pass Math": type_math_perc,
          "% Pass Read": type_read_perc,
          "% Overall Pass": type_overall_perc})

type_sum_df["Ave Math Score"] = type_sum_df["Ave Math Score"].map("{:.1f}".format)
type_sum_df["Ave Read Score"] = type_sum_df["Ave Read Score"].map("{:.1f}".format)
type_sum_df["% Pass Math"] = type_sum_df["% Pass Math"].map("{:.0f}".format)
type_sum_df["% Pass Read"] = type_sum_df["% Pass Read"].map("{:.0f}".format)
type_sum_df["% Overall Pass"] = type_sum_df["% Overall Pass"].map("{:.0f}".format)

#print output
type_sum_df

,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
School Type,,,,,
Charter,83.5,83.9,90,93,87
District,77.0,81.0,67,81,54
